In [2]:
from crypto import cryptopuf
import numpy as np
from pypuf.pypuf import io
from pypuf.pypuf.metrics import uniqueness, bias
from pypuf.pypuf import attack
from pypuf.pypuf import metrics

2024-11-13 14:07:25.963042: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-13 14:07:26.069443: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-13 14:07:26.115824: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-13 14:07:26.138764: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 14:07:26.221998: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# Uniqueness evaluation
seed = 31415
np.random.seed(seed)
puf_instances = []
for i in range(5):  # Create 5 PUF instances
    new_key = np.random.bytes(16).hex()  # New random key for each instance
    nonce = "000000000000000000000000"  # 96-bit nonce for TinyJAMBU
    crypto_puf = cryptopuf.CryptoPUF(challenge_length=128, key=new_key, nonce=nonce, crypto='tinyjambu', seed=seed)
    print(f"key is {new_key}")
    puf_instances.append(crypto_puf)

# Evaluate uniqueness using the `uniqueness()` function
uniqueness_score = uniqueness(puf_instances, seed=seed, N=1000)
print(f"Uniqueness score of the responses: {uniqueness_score}")

# Bias evaluation
bias_score = bias(puf_instances[0], seed=seed)
print(f"bias is {bias_score}")


Initializing CryptoPUF based on tinyjambu crypto core...
key is 3e5e54a5a71836fce336d8ac0d68ec14
Initializing CryptoPUF based on tinyjambu crypto core...
key is 3e5e54a5a71836fce336d8ac0d68ec14
Initializing CryptoPUF based on tinyjambu crypto core...
key is 3e5e54a5a71836fce336d8ac0d68ec14
Initializing CryptoPUF based on tinyjambu crypto core...
key is 3e5e54a5a71836fce336d8ac0d68ec14
Initializing CryptoPUF based on tinyjambu crypto core...
key is 3e5e54a5a71836fce336d8ac0d68ec14
Uniqueness score of the responses: [0.]
bias is -0.004


In [4]:
N = 50000
print(f"Training with {N} CRPs...")

# Generate CRPs for the current iteration
crps = io.ChallengeResponseSet.from_simulation(crypto_puf, N=N, seed=2)

# Create and train the MLPAttack202 attack with the generated CRPs
attack_instance = attack.LRAttack2021(crps, seed=3, k=4, bs=1000, lr=.001, epochs=100)
attack_instance.fit()

# Evaluate the model's similarity score
model = attack_instance.model
score = metrics.similarity(crypto_puf, model, seed=4)
print(f"Predication accuracy of the trained LR model is with 50000 CRPs: {score}")


Training with 50000 CRPs...
Epoch 1/100


I0000 00:00:1731524851.634409    3083 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:03:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-13 14:07:31.818842: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4988 - loss: 0.6938 - val_accuracy: 0.4800 - val_loss: 0.6947
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5146 - loss: 0.6921 - val_accuracy: 0.4940 - val_loss: 0.6953
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5200 - loss: 0.6912 - val_accuracy: 0.5180 - val_loss: 0.6957
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5195 - loss: 0.6904 - val_accuracy: 0.5260 - val_loss: 0.6960
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5248 - loss: 0.6895 - val_accuracy: 0.5320 - val_loss: 0.6959
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5270 - loss: 0.6886 - val_accuracy: 0.5300 - val_loss: 0.6956
Epoch 7/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 946us/step - accuracy: 0.5288 - loss: 0.6877 - val_accuracy: 0.5240 - val_loss: 0.6952
Epoch 8/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 964us/step - accuracy: 0.5314 - loss: 0.6869 - val_accuracy: 0.5240 - val_loss:

In [8]:
# Create and train the MLPAttack202 attack with the generated CRPs
attack_instance = attack.MLPAttack2021(
crps, seed=3, net=[2 ** 4, 2 ** 5, 2 ** 4],
epochs=30, lr=.001, bs=1000, early_stop=.08
)
attack_instance.fit()

# Evaluate the model's similarity score
model = attack_instance.model
score = metrics.similarity(crypto_puf, model, seed=4)
print(f"Predication accuracy of the trained LR model is with 50000 CRPs: {score}")

Epoch 1/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4938 - loss: 0.7201 - val_accuracy: 0.4820 - val_loss: 0.6954
Epoch 2/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5133 - loss: 0.6927 - val_accuracy: 0.4800 - val_loss: 0.6952
Epoch 3/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5201 - loss: 0.6917 - val_accuracy: 0.4900 - val_loss: 0.6961
Epoch 4/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5256 - loss: 0.6908 - val_accuracy: 0.5000 - val_loss: 0.6976
Epoch 5/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5331 - loss: 0.6900 - val_accuracy: 0.4980 - val_loss: 0.6994
Epoch 6/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5362 - loss: 0.6892 - val_accuracy: 0.4720 - val_loss: 0.7015
Epoch 7/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5386 - loss: 0.6883 - val_accuracy: 0.4620 - val_loss: 0.7038
Epoch 8/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5447 - loss: 0.6874 - val_accuracy: 0.4760 - val_loss: